<a href="https://colab.research.google.com/github/WuilsonEstacio/Procesamiento-de-lenguaje-natural/blob/main/procesamiento_de_lenguaje_natural_algorit_Viterbi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmo de Viterbi

Permite hallar la secuencia más probable de estados ocultos que produce una secuencia observada de sucesos, especialmente en el contexto de fuentes de información de Márkov y modelos ocultos de Márkov. Se aplica de forma general en la descodificación de códigos convolucionales usados en redes de telefonía celular digital GSM y CDMA, módems de líneas conmutadas, satélites, comunicaciones espaciales y redes inalámbricas IEEE 802.11. También se usa en reconocimiento del habla, síntesis de habla, diarización, búsqueda de palabras clave, lingüística computacional y bioinformática.
 https://es.wikipedia.org/wiki/Algoritmo_de_Viterbi 

https://github.com/rb-one/Curso_Algoritmos_Clasificacion_Texto/blob/main/Notas/notes.md

Estos pasos dependen de algoritmo creado en el cuaderno anterior.

procesamiento de lenguaje natural Etiquetado.ipynb

In [ ]:
# instalacion de dependencias previas
!pip install conllu
!git clone https://github.com/UniversalDependencies/UD_Spanish-AnCora.git

# Carga del modelo HMM previamente entrenado

In [ ]:
# cargamos las probabilidades del modelo HMM del codigo de emision y transmision dadas en 
# procesamiento de lenguaje natural Etiquetado.ipynb
import numpy as np
# se crean bariables para almacenas probabilidades de emision y transision 
transitionProbdict = np.load('transitionHMM.npy', allow_pickle='TRUE').item() # allow_pickle Permitir guardar matrices de objetos utilizando encurtidos Python
emissionProbdict = np.load('emissionHMM.npy', allow_pickle='TRUE').item()

In [ ]:
#  muestra las probabilidades condicionales
transitionProbdict
# emissionProbdict
# con esto ya hemos cargado el modelo

In [ ]:
# identificamos las categorias gramaticales 'upos' unicas en el corpus
# utilizamos set para evitar que se repitan las categorias gramaticales
stateSet = set([w.split('|')[1] for w in list(emissionProbdict.keys())]) # escogemos w.split('|') para solo selecional la categoria gramatical de cada uno de los elementos
stateSet

In [ ]:
# enumeramos las categorias con numeros para asignar a 
# las columnas de la matriz de Viterbi

# esto es un diccionario con relaciones llave valor donde las llaves 
# van ha ser las categorias gramaticales y los valores el numero de la clumna que le va a corresponder en la matriz
tagStateDict = {} 
for i, state in enumerate(stateSet): 
  tagStateDict[state] = i
tagStateDict
# con esto tenemos un diccionario que relaciona cada categoria gramatical
# con un indice que dice cual seria la fila que le corresponde dentro de la matriz de viterbi

# Distribucion inicial de estados latentes

los estados latentes son la primera palabra que hay en cada frase dl corpus

In [ ]:
# Calculamos distribución inicial de estados
initTagStateProb = {} # \rho_i^{(0)} creamos este diccionario que son las probabilidades de encontrar cierta categoria gramatical al inicio de la frase
from conllu import parse_incr 
wordList = []
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8")
count = 0 # cuenta la longitud del corpus
for tokenlist in parse_incr(data_file):
  count += 1 # incrementamos el contador
  tag = tokenlist[0]['upos'] # cojo el primer elemento con la catagoria gramatical con el upos
  if tag in initTagStateProb.keys():
    initTagStateProb[tag] += 1
  else:
    initTagStateProb[tag] = 1


for key in initTagStateProb.keys(): 
  initTagStateProb[key] /= count # a cada elemento del dicionatio lo divido por el total de frases que hay

initTagStateProb 
# esto nos devuelve las probabilidades por cada una de las categorias

In [ ]:
# para calcular la suma de las probabilidades que tiene que ser 1
sum(initTagStateProb.values()) 

In [ ]:
# verificamos que la suma de las probabilidades que deben sumar  1 (100%)
np.array([initTagStateProb[k] for k in initTagStateProb.keys()]).sum()

# Construccion Algoritmo de Viterbi

Dada una secuencia de palabras $\{p_1, p_2, \dots, p_n \}$, y un conjunto de categorias gramaticales dadas por la convención `upos`, se considera la matriz de probabilidades de Viterbi así:

$$
\begin{array}{c c}
\begin{array}{c c c c}
\text{ADJ} \\
\text{ADV}\\
\text{PRON} \\
\vdots \\
{}
\end{array} 
&
\left[
\begin{array}{c c c c}
\nu_1(\text{ADJ}) & \nu_2(\text{ADJ}) & \dots  & \nu_n(\text{ADJ})\\
\nu_1(\text{ADV}) & \nu_2(\text{ADV}) & \dots  & \nu_n(\text{ADV})\\ 
\nu_1(\text{PRON}) & \nu_2(\text{PRON}) & \dots  & \nu_n(\text{PRON})\\
\vdots & \vdots & \dots & \vdots \\ \hdashline
p_1 & p_2 & \dots & p_n 
\end{array}
\right] 
\end{array}
$$

Donde las probabilidades de la primera columna (para una categoria $i$) están dadas por: 

$$
\nu_1(i) = \underbrace{\rho_i^{(0)}}_{\text{probabilidad inicial}} \times \underbrace{P(p_1 \vert i)}_{\text{emisión}}
$$

luego, para la segunda columna (dada una categoria $j$) serán: 

$$
\nu_2(j) = \max_i \{ \nu_1(i) \times \underbrace{P(j \vert i)}_{\text{transición}} \times \underbrace{P(p_2 \vert j)}_{\text{emisión}} \}
$$

así, en general las probabilidades para la columna $t$ estarán dadas por: 

$$
\nu_{t}(j) = \max_i \{ \overbrace{\nu_{t-1}(i)}^{\text{estado anterior}} \times \underbrace{P(j \vert i)}_{\text{transición}} \times \underbrace{P(p_t \vert j)}_{\text{emisión}} \}
$$

In [ ]:
import nltk 
nltk.download('punkt')
from nltk import word_tokenize

In [ ]:
# debemos construir la funcion porque el resultado de la funcio sera devolverme la matriz de Vitervi 
# que corresponde a la secuencia de palabras que le vamos a pasar como argumento de entrada a ala funcion
def ViterbiMatrix(secuencia, transitionProbdict=transitionProbdict,
                  emissionProbdict=emissionProbdict, tagStateDict=tagStateDict,
                  initTagStateProb=initTagStateProb):
  seq = word_tokenize(secuencia) # creamos el toquenizador para una secuencia toquenizada
  viterbiProb = np.zeros((17, len(seq)))  # 17 porque upos tiene 17 categorias, matriz(filas,columnas)

  # inicialización primera columna
  for key in tagStateDict.keys():
    tag_row = tagStateDict[key] # generemos una variable de la fila corresondiente a la etiqueta
    word_tag = seq[0].lower()+'|'+key # asignamos de la secuencia e palabras el primer elemento y sumo la key que coresponde a al etiqueta 
    if word_tag in emissionProbdict.keys(): # creamos los elementos de la matriz
      viterbiProb[tag_row, 0] = initTagStateProb[key]*emissionProbdict[word_tag]

  # computo de todas las probabilidades de las siguientes columnas
  for col in range(1, len(seq)): # col para recores las columnas
    for key in tagStateDict.keys(): # hago un for sobre cada llave o categorias gramatica
      tag_row = tagStateDict[key] # asigno la fila relacionada con la categoria gramatical
      word_tag = seq[col].lower()+'|'+key
      if word_tag in emissionProbdict.keys(): # para ver si la etiqueta palabra esta en emission p..
        # miramos estados de la col anterior
        possible_probs = [] # para ello  creamos lista basia que la llenaremos con un for
        for key2 in tagStateDict.keys(): 
          tag_row2 = tagStateDict[key2] # creamos la fila asociada a key2
          tag_prevtag = key+'|'+key2 # consideramos las probabilidades de transiscion y eso se hace ju tando la llave actual con la anterior
          if tag_prevtag in transitionProbdict.keys():
            if viterbiProb[tag_row2, col-1]>0: # aqui y en las 2 siguentes lineas consideramos la ultima parte 
              possible_probs.append(
                  viterbiProb[tag_row2, col-1]*transitionProbdict[tag_prevtag]*emissionProbdict[word_tag])
        viterbiProb[tag_row, col] = max(possible_probs)  # ahora debemos escoger el maximo de todos esos elementos
  
  return viterbiProb

matrix = ViterbiMatrix('El mundo es pequeño')
matrix

In [ ]:
# funcion para calcular las etiquetas
def ViterbiTags(secuencia, transitionProbdict=transitionProbdict, emissionProbdict=emissionProbdict, 
            tagStateDict=tagStateDict, initTagStateProb=initTagStateProb):
  seq = word_tokenize(secuencia)
  viterbiProb = np.zeros((17, len(seq)))  # upos tiene 17 categorias

  # inicialización primera columna
  for key in tagStateDict.keys():
    tag_row = tagStateDict[key]
    word_tag = seq[0].lower()+'|'+key
    if word_tag in emissionProbdict.keys():
      viterbiProb[tag_row, 0] = initTagStateProb[key]*emissionProbdict[word_tag]

  # computo de las siguientes columnas
  for col in range(1, len(seq)):
    for key in tagStateDict.keys():
      tag_row = tagStateDict[key]
      word_tag = seq[col].lower()+'|'+key
      if word_tag in emissionProbdict.keys():
        # miramos estados de la col anterior
        possible_probs = []
        for key2 in tagStateDict.keys(): 
          tag_row2 = tagStateDict[key2]
          tag_prevtag = key+'|'+key2
          if tag_prevtag in transitionProbdict.keys():
            if viterbiProb[tag_row2, col-1]>0:
              possible_probs.append(
                  viterbiProb[tag_row2, col-1]*transitionProbdict[tag_prevtag]*emissionProbdict[word_tag])
        viterbiProb[tag_row, col] = max(possible_probs)
# hasta aqui repetimos el paso anterior

# ahora agregaremos un paso extra donde realizaremos
    # contruccion de secuencia de tags o etiquetas
    res = [] # contruimos lista basia llamada res
    for i, p in enumerate(seq): # creamos for para recorrer cada palabra de la secuencia y ademas la vamos enumerando las palabras
      for tag in tagStateDict.keys(): #para recorrer todos los tags para que pueda ver la probabilidad de que esa palabra este asociada a lasetiquetas
        if tagStateDict[tag] == np.argmax(viterbiProb[:, i]):
          res.append((p, tag)) # este paso es para  agregar lo anterior a la lista
                                # aqui tendreos una lista de palabra etiqueta
  return res

ViterbiTags('el mundo es muy pequeño')

In [ ]:
ViterbiTags('estos instrumentos han de rasgar')

# Entrenamiento directo de HMM con NLTK
clase en python (NLTK) de HMM: https://www.nltk.org/_modules/nltk/tag/hmm.html

# Ejemplo con el Corpus Treebank en ingles


In [ ]:
#@title ejemplo con el Corpus Treebank en ingles
import nltk
nltk.download('treebank')
from nltk.corpus import treebank
train_data = treebank.tagged_sents()[:3900] # esto es asi para coger una parte de los datos para entrenamiento y la otra parte para prueba
# y escogemos hasta la sentencia 3900 y dejamos el resto para test

# estructura de la data de entrenamiento

In [ ]:
#@title estructura de la data de entrenamiento
train_data

# HMM pre-construido en NLTK


In [ ]:
#@title HMM pre-construido en NLTK
from nltk.tag import hmm # cargamos clase para poder utilizar modelo
tagger = hmm.HiddenMarkovModelTrainer().train_supervised(train_data) # parapedir que sea un entrenamiento supervisado
# tagger = hmm.HiddenMarkovModelTagger.train(train_data) 
tagger

In [ ]:
tagger.tag("Pierre Vinken will get old".split()) # aquii estamos tokenizando con un .split()

In [ ]:
#@title training accuracy
# para ver el porcentaje de evaluacion del modelos, osea su presicion sobre el conjunto de entrenamiento
tagger.evaluate(treebank.tagged_sents()[:3900])

Ejercicio de práctica
Objetivo: Entrena un HMM usando la clase hmm.HiddenMarkovModelTrainer() sobre el dataset UD_Spanish_AnCora.

1. **Pre-procesamiento:** En el ejemplo anterior usamos el dataset en ingles `treebank`, el cual viene con una estructura diferente a la de `AnCora`, en esta parte escribe código para transformar la estructura de `AnCora` de manera que quede igual al `treebank` que usamos así:

$$\left[ \left[ (\text{'El'}, \text{'DET'}), (\dots), \dots\right], \left[\dots \right] \right]$$

In [ ]:
# desarrolla tu código aquí 
# Instalamos conllu para leer el corpus
!pip install conllu
# Bajamos el corpus de AnCora
!git clone https://github.com/UniversalDependencies/UD_Spanish-AnCora.git
from conllu import parse_incr
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8") 


# Hacemos la transformacion del corpus al formato requerido
wordList = [] # creamos lista basia
for tokenlist in parse_incr(data_file): 
  wordList2 = []
  for token in tokenlist:
    tag = token['upos']
    valor = token['form']
    wordList2.append((valor,tag)) 
  wordList.append(wordList2)

2. **Entrenamiento:** Una vez que el dataset esta con la estructura correcta, utiliza la clase `hmm.HiddenMarkovModelTrainer()` para entrenar con el $80 \%$ del dataset como conjunto de `entrenamiento` y $20 \%$ para el conjunto de `test`.

**Ayuda:** Para la separacion entre conjuntos de entrenamiento y test, puedes usar la funcion de Scikit Learn: 

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

En este punto el curso de Machine Learning con Scikit Learn es un buen complemento para entender mejor las funcionalidades de Scikit Learn: https://platzi.com/cursos/scikitlearn-ml/ 

In [ ]:
# desarrolla tu código aquí

import nltk
from nltk.tag import hmm
from sklearn.model_selection import train_test_split

3. **Validación del modelo:** Un vez entrenado el `tagger`, calcula el rendimiento del modelo (usando `tagger.evaluate()`) para los conjuntos de `entrenamiento` y `test`.


In [ ]:
#desarrolla tu código aquí
# Separamos el corpus
wordList_train, wordList_test= train_test_split(wordList, test_size=0.20, random_state=42)

# Entrenamos el modelo
tagger = hmm.HiddenMarkovModelTrainer().train_supervised(wordList_train)
tagger

print(tagger.evaluate(wordList_test))
print(tagger.evaluate(wordList_train))

Procesamiento de lenguaje natural
kERNEL: es una funcion matematica que toma mediciones que se comportan de manera no lineal